In [2]:
import os 
import pandas as pd
from tqdm import tqdm
from collections import Counter, defaultdict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef

path_benchmark = "/media/concha-eloko/Linux/PPT_clean/benchmark"


***
### Matrice results: 

In [115]:
# TropiGAT
tropigat_pred_m_df = pd.read_csv(f"{path_benchmark}/TropiGAT.matrices.tsv", sep = "\t", names = ["protein", "prediction"])
tropigat_pred_m_df["protein_id"] = tropigat_pred_m_df["protein"].apply(lambda x : x.split("_Dpo")[0])
tropigat_pred_m_df.columns = ["protein_TropiGAT", "prediction_TropiGAT", "protein_id"]

tropigat_pred_m_df_uf = pd.read_csv(f"{path_benchmark}/TropiGAT_UF.review.matrices.tsv", sep = "\t", names = ["protein", "prediction"])
tropigat_pred_m_df_uf["protein_id"] = tropigat_pred_m_df_uf["protein"].apply(lambda x : x.split("_Dpo")[0])
tropigat_pred_m_df_uf.columns = ["protein_TropiGAT", "prediction_TropiGAT_UF", "protein_id"]
tropigat_pred_m_df_uf = tropigat_pred_m_df_uf[["prediction_TropiGAT_UF", "protein_id"]]


tropigat_pred_m_df_new = pd.read_csv(f"{path_benchmark}/TropiGAT.review.matrices.tsv", sep = "\t", names = ["protein", "prediction"])
tropigat_pred_m_df_new["protein_id"] = tropigat_pred_m_df_new["protein"].apply(lambda x : x.split("_Dpo")[0])
tropigat_pred_m_df_new.columns = ["protein_TropiGAT_new", "prediction_TropiGAT_new", "protein_id"]
tropigat_pred_m_df_new = tropigat_pred_m_df_new[["prediction_TropiGAT_new", "protein_id"]]

# TropiSAGE
tropisage_pred_m_df = pd.read_csv(f"{path_benchmark}/TropiSAGE.review.matrices.tsv", sep = "\t", names = ["protein", "prediction"])
tropisage_pred_m_df["protein_id"] = tropisage_pred_m_df["protein"].apply(lambda x : x.split("_Dpo")[0])
tropisage_pred_m_df.columns = ["protein_TropiSAGE", "prediction_TropiSAGE", "protein_id"]
tropisage_pred_m_df = tropisage_pred_m_df[["prediction_TropiSAGE", "protein_id"]]

# TropiSEQ
tropiseq_pred_m_df = pd.read_csv(f"{path_benchmark}/TropiSEQ.p_05.matrices.tsv", sep = "\t", names = ["protein", "prediction"])
tropiseq_pred_m_df["protein_id_1"] = tropiseq_pred_m_df["protein"].apply(lambda x : x.split("_A")[0] if x.count("_A")>0 else "__".join(x.split(",")[0].split(" ")) if x.count(",")>0 else x)
tropiseq_pred_m_df["protein_id"] = tropiseq_pred_m_df["protein_id_1"].apply(lambda x : x.replace(" cds", "__cds"))
tropiseq_pred_m_df.columns = ["protein_TropiSEQ", "prediction_TropiSEQ", "trash_seq" ,"protein_id"]
tropiseq_pred_m_df = tropiseq_pred_m_df[["prediction_TropiSEQ" ,"protein_id"]]


# TropiSEQ UF
tropiseq_pred_m_df_uf = pd.read_csv(f"{path_benchmark}/TropiSEQ_UF_70.p_05.matrices.tsv", sep = "\t", names = ["protein", "prediction"])
tropiseq_pred_m_df_uf["protein_id_1"] = tropiseq_pred_m_df_uf["protein"].apply(lambda x : x.split("_A")[0] if x.count("_A")>0 else "__".join(x.split(",")[0].split(" ")) if x.count(",")>0 else x)
tropiseq_pred_m_df_uf["protein_id"] = tropiseq_pred_m_df_uf["protein_id_1"].apply(lambda x : x.replace(" cds", "__cds"))
tropiseq_pred_m_df_uf.columns = ["protein_TropiSEQ", "prediction_TropiSEQ_UF", "trash_seq" ,"protein_id"]
tropiseq_pred_m_df_uf = tropiseq_pred_m_df_uf[["prediction_TropiSEQ_UF" ,"protein_id"]]


# TropiLR
tropilr_pred_m_df = pd.read_csv(f"{path_benchmark}/TropiLogReg.matrices.tsv", sep = "\t", names = ["protein", "prediction"])
tropilr_pred_m_df["protein_id_1"] = tropilr_pred_m_df["protein"].apply(lambda x : x.split("_A")[0] if x.count("_A")>0 else "__".join(x.split(",")[0].split(" ")) if x.count(",")>0 else x)
tropilr_pred_m_df["protein_id"] = tropilr_pred_m_df["protein_id_1"].apply(lambda x : x.replace(" cds", "__cds"))
tropilr_pred_m_df.columns = ["protein_TropiLR", "prediction_TropiLR", "trash_seq" ,"protein_id"]
tropilr_pred_m_df = tropilr_pred_m_df[["prediction_TropiLR" ,"protein_id"]]

# SpikeHunter
SH_pred_m_df = pd.read_csv(f"{path_benchmark}/SpikeHunter_predictions.matrices.tsv", sep = "\t", names = ["protein", "prediction"])

SH_pred_m_df["protein_id_1"] = SH_pred_m_df["protein"].apply(lambda x : x.split("_A")[0] if x.count("_A")>0 else "__".join(x.split(",")[0].split(" ")) if x.count(",")>0 else x)
SH_pred_m_df["protein_id"] = SH_pred_m_df["protein_id_1"].apply(lambda x : x.replace(" cds", "__cds"))
SH_pred_m_df.columns = ["protein_SpikeHunter", "prediction_SpikeHunter", "trash_seq" ,"protein_id"]
SH_pred_m_df = SH_pred_m_df[["prediction_SpikeHunter" ,"protein_id"]]


# Merge : 
#merged_df = tropigat_pred_m_df.merge(tropiseq_pred_m_df, on='protein_id', how='inner').merge(SH_pred_m_df, on='protein_id', how='inner').merge(tropigat_pred_m_df_new, on='protein_id', how='inner').merge(tropisage_pred_m_df, on='protein_id', how='inner').merge(tropilr_pred_m_df, on='protein_id', how='inner')
merged_df = tropigat_pred_m_df.merge(tropiseq_pred_m_df, on='protein_id', how='inner').merge(tropigat_pred_m_df_uf, on='protein_id', how='inner').merge(tropiseq_pred_m_df_uf, on='protein_id', how='inner').merge(SH_pred_m_df, on='protein_id', how='inner').merge(tropigat_pred_m_df_new, on='protein_id', how='inner').merge(tropisage_pred_m_df, on='protein_id', how='inner').merge(tropilr_pred_m_df, on='protein_id', how='inner')
merged_df["phage"] = merged_df["protein_id"].apply(lambda x : x.split("__")[0] if x.count("__")>0 else x.split("_")[0])


In [116]:
merged_df

,protein_TropiGAT,prediction_TropiGAT,protein_id,prediction_TropiSEQ,prediction_TropiGAT_UF,prediction_TropiSEQ_UF,prediction_SpikeHunter,prediction_TropiGAT_new,prediction_TropiSAGE,prediction_TropiLR,phage
0,K15PH90__cds_54_Dpo_domain,KL123:0.997 ; KL27:0.988 ; KL112:0.985 ; KL14:...,K15PH90__cds_54,No_hits,KL64:0.997 ; KL23:0.986 ; KL67:0.984 ; KL26:0....,No_hits,K106,KL112:0.988 ; KL166:0.968 ; KL8:0.953 ; KL38:0...,KL166:1.0 ; KL112:1.0 ; KL8:0.999 ; KL124:0.99...,No_hits,K15PH90
1,K7PH164C4__cds_20_Dpo_domain,KL39:1.0 ; KL123:0.999 ; KL22:0.996 ; KL114:0....,K7PH164C4__cds_20,No_hits,KL26:1.0 ; KL106:1.0 ; KL149:0.999 ; KL105:0.9...,No_hits,No_hits,KL105:1.0 ; KL149:0.999 ; KL22:0.998 ; KL106:0...,KL47:1.0 ; KL116:1.0 ; KL105:1.0 ; KL39:1.0 ; ...,No_hits,K7PH164C4
2,K32PH164C1__cds_20_Dpo_domain,KL36:1.0 ; KL39:0.998 ; KL25:0.992 ; KL116:0.9...,K32PH164C1__cds_20,No_hits,KL23:1.0 ; KL46:0.999 ; KL26:0.994 ; KL24:0.98...,No_hits,No_hits,KL36:1.0 ; KL105:0.998 ; KL8:0.998 ; KL22:0.99...,KL36:1.0 ; KL166:1.0 ; KL46:1.0 ; KL105:1.0 ; ...,No_hits,K32PH164C1
3,K18PH07C1__cds_245_Dpo_domain,KL3:0.998 ; KL63:0.984 ; KL43:0.94 ; KL145:0.9...,K18PH07C1__cds_245,KL63:0.594 ; KL37:0.235 ; KL164:0.195 ; KL61:0...,KL63:0.967 ; KL6:0.829 ; KL15:0.608 ; KL26:0.594,KL6:1.0 ; KL63:0.494 ; KL4:0.168 ; KL117:0.168...,K63,KL3:0.93 ; KL63:0.872 ; KL70:0.768 ; KL54:0.64...,KL6:1.0 ; KL63:0.999 ; KL26:0.988 ; KL31:0.974...,KL6:0.523 ; KL63:0.682,K18PH07C1
4,K13PH07C1L__cds_11_Dpo_domain,KL23:0.992 ; KL10:0.99 ; KL30:0.975 ; KL60:0.9...,K13PH07C1L__cds_11,KL13:0.527 ; KL2:0.417 ; KL37:0.235 ; KL164:0....,KL60:0.941 ; KL71:0.88 ; KL122:0.814 ; KL107:0...,KL2:0.409 ; KL6:0.213 ; KL13:0.201 ; KL4:0.168...,K2,KL12:0.999 ; KL128:0.991 ; KL105:0.962 ; KL36:...,KL105:1.0 ; KL128:0.995 ; KL30:0.995 ; KL36:0....,No_associations,K13PH07C1L
...,...,...,...,...,...,...,...,...,...,...,...
267,PFOEGONH_00078,KL3:1.0 ; KL81:0.985 ; KL14:0.957 ; KL52:0.857...,PFOEGONH_00078,KL35:0.736 ; KL3:0.674 ; KL124:0.332 ; KL37:0....,KL3:0.998 ; KL169:0.931 ; KL35:0.913,KL3:0.615 ; KL35:0.385 ; KL6:0.213 ; KL30:0.17...,K3,KL3:0.999 ; KL117:0.951 ; KL31:0.789 ; KL52:0....,KL3:1.0 ; KL67:0.99 ; KL14:0.862 ; KL52:0.794 ...,KL35:0.71 ; KL3:0.947,PFOEGONH
268,NBNDMPCG_00163,KL13:0.958 ; KL2:0.954 ; KL57:0.94 ; KL60:0.92...,NBNDMPCG_00163,KL2:0.684 ; KL52:0.239 ; KL37:0.235 ; KL164:0....,KL2:0.937 ; KL18:0.842 ; KL71:0.823 ; KL12:0.773,KL2:0.728 ; KL52:0.557 ; KL6:0.213 ; KL13:0.17...,K2,KL12:0.998 ; KL13:0.929 ; KL9:0.92 ; KL2:0.652...,KL12:0.999 ; KL9:0.99 ; KL13:0.981 ; KL142:0.9...,KL2:0.829 ; KL52:0.774,NBNDMPCG
269,NJHLHPIG_00061,KL46:0.99 ; KL128:0.975 ; KL18:0.971 ; KL52:0....,NJHLHPIG_00061,KL46:0.937 ; KL18:0.522 ; KL37:0.235 ; KL164:0...,KL46:0.999 ; KL128:0.991 ; KL117:0.96 ; KL17:0...,KL46:0.668 ; KL18:0.322 ; KL6:0.213 ; KL4:0.16...,K46,KL128:0.999 ; KL46:0.999 ; KL12:0.984 ; KL56:0...,KL46:0.999 ; KL56:0.998 ; KL128:0.984 ; KL26:0...,KL46:0.82,NJHLHPIG
270,HIIECEMK_00054,KL60:0.995 ; KL18:0.991 ; KL23:0.943 ; KL14:0....,HIIECEMK_00054,KL13:0.527 ; KL2:0.417 ; KL37:0.235 ; KL164:0....,KL45:0.951 ; KL60:0.927 ; KL2:0.866 ; KL18:0.8...,KL2:0.409 ; KL6:0.213 ; KL13:0.201 ; KL4:0.168...,K2,KL12:0.991 ; KL18:0.976 ; KL60:0.93 ; KL70:0.7...,KL8:0.999 ; KL12:0.998 ; KL18:0.997 ; KL142:0....,No_associations,HIIECEMK


In [117]:
missing_prot = [prot for prot in list(tropigat_pred_m_df["protein_id"]) if prot not in list(SH_pred_m_df["protein_id"])]
missing_prot_prime = [prot for prot in list(SH_pred_m_df["protein_id"]) if prot not in list(tropigat_pred_m_df["protein_id"])]

In [118]:
missing_prot

['BLCJPOBP_00052']

In [7]:
missing_prot_prime

['K33PH14C2__cds_55', 'K62PH164C2__cds_25']

In [8]:
len(tropigat_pred_m_df), len(tropiseq_pred_m_df),len(SH_pred_m_df), len(merged_df)

(258, 260, 260, 272)

In [9]:
merged_df.columns

Index(['protein_TropiGAT', 'prediction_TropiGAT', 'protein_id',
       'prediction_TropiSEQ', 'prediction_TropiGAT_UF',
       'prediction_TropiSEQ_UF', 'prediction_SpikeHunter',
       'prediction_TropiGAT_new', 'prediction_TropiSAGE', 'prediction_TropiLR',
       'phage'],
      dtype='object')

In [119]:
phages_to_eliminate = ["K80PH1317b", "K80PH1317a"] + ["K2064PH2","K2069PH1","OBHDAGOG","A1e","A1f","A3a","EONHMLJF"]
merged_df_clean_01 = merged_df[["phage", "protein_id", "prediction_TropiGAT", "prediction_TropiGAT_new", "prediction_TropiGAT_UF" ,"prediction_TropiSAGE", "prediction_TropiSEQ", "prediction_TropiSEQ_UF","prediction_TropiLR", "prediction_SpikeHunter"]]

#merged_df_clean_01.columns = ["phage", "protein", "TropiGAT_prediction", "TropiSEQ_prediction", "SpikeHunter_prediction"]
merged_df_clean_01 = merged_df_clean_01[~merged_df_clean_01["phage"].isin(phages_to_eliminate)] 
merged_df_clean_01

,phage,protein_id,prediction_TropiGAT,prediction_TropiGAT_new,prediction_TropiGAT_UF,prediction_TropiSAGE,prediction_TropiSEQ,prediction_TropiSEQ_UF,prediction_TropiLR,prediction_SpikeHunter
0,K15PH90,K15PH90__cds_54,KL123:0.997 ; KL27:0.988 ; KL112:0.985 ; KL14:...,KL112:0.988 ; KL166:0.968 ; KL8:0.953 ; KL38:0...,KL64:0.997 ; KL23:0.986 ; KL67:0.984 ; KL26:0....,KL166:1.0 ; KL112:1.0 ; KL8:0.999 ; KL124:0.99...,No_hits,No_hits,No_hits,K106
1,K7PH164C4,K7PH164C4__cds_20,KL39:1.0 ; KL123:0.999 ; KL22:0.996 ; KL114:0....,KL105:1.0 ; KL149:0.999 ; KL22:0.998 ; KL106:0...,KL26:1.0 ; KL106:1.0 ; KL149:0.999 ; KL105:0.9...,KL47:1.0 ; KL116:1.0 ; KL105:1.0 ; KL39:1.0 ; ...,No_hits,No_hits,No_hits,No_hits
2,K32PH164C1,K32PH164C1__cds_20,KL36:1.0 ; KL39:0.998 ; KL25:0.992 ; KL116:0.9...,KL36:1.0 ; KL105:0.998 ; KL8:0.998 ; KL22:0.99...,KL23:1.0 ; KL46:0.999 ; KL26:0.994 ; KL24:0.98...,KL36:1.0 ; KL166:1.0 ; KL46:1.0 ; KL105:1.0 ; ...,No_hits,No_hits,No_hits,No_hits
3,K18PH07C1,K18PH07C1__cds_245,KL3:0.998 ; KL63:0.984 ; KL43:0.94 ; KL145:0.9...,KL3:0.93 ; KL63:0.872 ; KL70:0.768 ; KL54:0.64...,KL63:0.967 ; KL6:0.829 ; KL15:0.608 ; KL26:0.594,KL6:1.0 ; KL63:0.999 ; KL26:0.988 ; KL31:0.974...,KL63:0.594 ; KL37:0.235 ; KL164:0.195 ; KL61:0...,KL6:1.0 ; KL63:0.494 ; KL4:0.168 ; KL117:0.168...,KL6:0.523 ; KL63:0.682,K63
4,K13PH07C1L,K13PH07C1L__cds_11,KL23:0.992 ; KL10:0.99 ; KL30:0.975 ; KL60:0.9...,KL12:0.999 ; KL128:0.991 ; KL105:0.962 ; KL36:...,KL60:0.941 ; KL71:0.88 ; KL122:0.814 ; KL107:0...,KL105:1.0 ; KL128:0.995 ; KL30:0.995 ; KL36:0....,KL13:0.527 ; KL2:0.417 ; KL37:0.235 ; KL164:0....,KL2:0.409 ; KL6:0.213 ; KL13:0.201 ; KL4:0.168...,No_associations,K2
...,...,...,...,...,...,...,...,...,...,...
267,PFOEGONH,PFOEGONH_00078,KL3:1.0 ; KL81:0.985 ; KL14:0.957 ; KL52:0.857...,KL3:0.999 ; KL117:0.951 ; KL31:0.789 ; KL52:0....,KL3:0.998 ; KL169:0.931 ; KL35:0.913,KL3:1.0 ; KL67:0.99 ; KL14:0.862 ; KL52:0.794 ...,KL35:0.736 ; KL3:0.674 ; KL124:0.332 ; KL37:0....,KL3:0.615 ; KL35:0.385 ; KL6:0.213 ; KL30:0.17...,KL35:0.71 ; KL3:0.947,K3
268,NBNDMPCG,NBNDMPCG_00163,KL13:0.958 ; KL2:0.954 ; KL57:0.94 ; KL60:0.92...,KL12:0.998 ; KL13:0.929 ; KL9:0.92 ; KL2:0.652...,KL2:0.937 ; KL18:0.842 ; KL71:0.823 ; KL12:0.773,KL12:0.999 ; KL9:0.99 ; KL13:0.981 ; KL142:0.9...,KL2:0.684 ; KL52:0.239 ; KL37:0.235 ; KL164:0....,KL2:0.728 ; KL52:0.557 ; KL6:0.213 ; KL13:0.17...,KL2:0.829 ; KL52:0.774,K2
269,NJHLHPIG,NJHLHPIG_00061,KL46:0.99 ; KL128:0.975 ; KL18:0.971 ; KL52:0....,KL128:0.999 ; KL46:0.999 ; KL12:0.984 ; KL56:0...,KL46:0.999 ; KL128:0.991 ; KL117:0.96 ; KL17:0...,KL46:0.999 ; KL56:0.998 ; KL128:0.984 ; KL26:0...,KL46:0.937 ; KL18:0.522 ; KL37:0.235 ; KL164:0...,KL46:0.668 ; KL18:0.322 ; KL6:0.213 ; KL4:0.16...,KL46:0.82,K46
270,HIIECEMK,HIIECEMK_00054,KL60:0.995 ; KL18:0.991 ; KL23:0.943 ; KL14:0....,KL12:0.991 ; KL18:0.976 ; KL60:0.93 ; KL70:0.7...,KL45:0.951 ; KL60:0.927 ; KL2:0.866 ; KL18:0.8...,KL8:0.999 ; KL12:0.998 ; KL18:0.997 ; KL142:0....,KL13:0.527 ; KL2:0.417 ; KL37:0.235 ; KL164:0....,KL2:0.409 ; KL6:0.213 ; KL13:0.201 ; KL4:0.168...,No_associations,K2


> Add the target data: 

In [120]:
# Fold data
path_project = "/media/concha-eloko/Linux/PPT_clean"
df_folds = pd.read_csv(f"{path_project}/in_vitro/dpos_folds.all_matrices.tsv", header = 0, sep = "\t")

# Target data
path_finetuning = "/media/concha-eloko/Linux/PPT_clean/in_vitro/fine_tuning"
bea_df = pd.read_csv(f"{path_finetuning}/bea_fine_tuning.df", sep = "\t", header = 0)
bea_df["Protein"] = bea_df["Protein"].apply(lambda x : x.replace("_", "__"))
pool_bea = set([kltype.strip() for kltypes in bea_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0])

ferriol_df = pd.read_csv(f"{path_finetuning}/ferriol_fine_tuning.df", sep = "\t", header = 0)
#ferriol_df["Target"] = ferriol_df["Target"].apply(lambda x : x.replace("K", "KL"))
pool_ferriol = set([kltype.strip() for kltypes in ferriol_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0])

towndsend_df = pd.read_csv(f"{path_finetuning}/towndsend_fine_tuning.df", sep = "\t", header = 0)
towndsend_df["Protein"] = towndsend_df["Protein"].apply(lambda x : x.replace("_", "__"))
pool_towndsend = set([kltype.strip() for kltypes in towndsend_df["Target"] for kltype in kltypes.split(",") if kltype.count("wzi") == 0 if kltype.count("pass") == 0])

dico_matrices = {"ferriol" : {"matrix" : ferriol_df, "pool" : pool_ferriol}, 
                 "bea" : {"matrix": bea_df, "pool" : pool_bea}, 
                 "towndsend" : {"matrix" : towndsend_df, "pool" : pool_towndsend}}

# targets dico : 
dico_hits = {}
for author in dico_matrices :
    matrix = dico_matrices[author]["matrix"]
    for _, row in matrix.iterrows() : 
        for phage in matrix["Phages"].unique() : 
            all_targets = set()
            targets = matrix[matrix["Phages"] == phage]["Target"].values
            for calls in targets : 
                actual_targets = [x.strip() for x in calls.split(",")]
                all_targets.update(actual_targets)
            dico_hits[phage] = all_targets


In [122]:
# Initialize new columns :
target_serie = []
fold_serie = []
tropigat_pred_serie, tropigatnew_pred_serie, tropiseq_pred_serie, tropiseq_pred_serie_uf ,tropigatnew_pred_serie_uf , sh_pred_serie, tropilr_pred_serie, tropisage_pred_serie = [], [], [] , [], [] , [], [], []

# Feed comumns : 
for _, row in merged_df_clean_01.iterrows() :
    if row["phage"] not in dico_hits :
        print(f"weird phage: {row['phage']}")
    else :
        targets = dico_hits[row["phage"]]
        target_serie.append(",".join(targets))
        prot_it = row['protein_id'].replace("__cds", "_cds")
        try :
            fold = df_folds[df_folds["protein_id"] == prot_it]["Fold"].values[0]
        except IndexError  :
            fold = df_folds[df_folds["protein_id"] == row['protein_id']]["Fold"].values[0]
        except Exception as e :
            fold = "unknown"
        fold_serie.append(fold)
        
        # TropiGAT part : 
        tropigat_pred = set([x.split(":")[0].strip() for x in row["prediction_TropiGAT"].split(";")])
        good_calls = tropigat_pred.intersection(targets)
        if len(good_calls) > 0 : 
            tropigat_pred_serie.append(",".join(list(good_calls)))
        else : 
            tropigat_pred_serie.append(0)
            
        # TropiGAT_new part : 
        tropigat_pred = set([x.split(":")[0].strip() for x in row["prediction_TropiGAT_new"].split(";")])
        good_calls = tropigat_pred.intersection(targets)
        if len(good_calls) > 0 : 
            tropigatnew_pred_serie.append(",".join(list(good_calls)))
        else : 
            tropigatnew_pred_serie.append(0)
            
        # TropiGAT_new_UF part : 
        tropigat_pred = set([x.split(":")[0].strip() for x in row["prediction_TropiGAT_UF"].split(";")])
        good_calls = tropigat_pred.intersection(targets)
        if len(good_calls) > 0 : 
            tropigatnew_pred_serie_uf.append(",".join(list(good_calls)))
        else : 
            tropigatnew_pred_serie_uf.append(0)
            
        # TropiSAGE part : 
        tropigat_pred = set([x.split(":")[0].strip() for x in row["prediction_TropiSAGE"].split(";")])
        good_calls = tropigat_pred.intersection(targets)
        if len(good_calls) > 0 : 
            tropisage_pred_serie.append(",".join(list(good_calls)))
        else : 
            tropisage_pred_serie.append(0)
            
        # TropiSEQ part : 
        if row["prediction_TropiSEQ"][0] != "N" : 
            tropiseq_pred = [x.split(":")[0].strip() for x in row["prediction_TropiSEQ"].split(";")]
            top_predictions = set(tropiseq_pred)
            good_calls = top_predictions.intersection(targets)
            if len(good_calls) > 0 : 
                tropiseq_pred_serie.append(",".join(list(good_calls)))
            else :
                tropiseq_pred_serie.append(0)
        else :
            tropiseq_pred_serie.append(0)
            
        # TropiSEQ UF  part : 
        if row["prediction_TropiSEQ"][0] != "N" : 
            tropiseq_pred = [x.split(":")[0].strip() for x in row["prediction_TropiSEQ_UF"].split(";")]
            top_predictions = set(tropiseq_pred)
            good_calls = top_predictions.intersection(targets)
            if len(good_calls) > 0 : 
                tropiseq_pred_serie_uf.append(",".join(list(good_calls)))
            else :
                tropiseq_pred_serie_uf.append(0)
        else :
            tropiseq_pred_serie_uf.append(0)
            
        # TropiLR part : 
        if row["prediction_TropiLR"][0] != "N" : 
            tropiseq_pred = [x.split(":")[0].strip() for x in row["prediction_TropiLR"].split(";")]
            top_predictions = set(tropiseq_pred)
            good_calls = top_predictions.intersection(targets)
            if len(good_calls) > 0 : 
                tropilr_pred_serie.append(",".join(list(good_calls)))
            else :
                tropilr_pred_serie.append(0)
        else :
            tropilr_pred_serie.append(0)
            
        # SH part : 
        if row["prediction_SpikeHunter"][0] != "N" : 
            sh_pred = [x.split(":")[0].strip().replace("K", "KL") for x in row["prediction_SpikeHunter"].split(";")]
            top_predictions = set(sh_pred)
            good_calls = top_predictions.intersection(targets)
            if len(good_calls) > 0 : 
                sh_pred_serie.append(",".join(list(good_calls)))
            else :
                sh_pred_serie.append(0)
        else :
            sh_pred_serie.append(0)
        
    

In [123]:
merged_df_clean_01["TropiGAT_good_calls"] = tropigat_pred_serie
merged_df_clean_01["TropiGAT_new_good_calls"] = tropigatnew_pred_serie
merged_df_clean_01["TropiGAT_new_UF_good_calls"] = tropigatnew_pred_serie_uf

merged_df_clean_01["TropiSAGE_good_calls"] = tropisage_pred_serie

merged_df_clean_01["TropiSEQ_good_calls"] = tropiseq_pred_serie
merged_df_clean_01["TropiSEQ_UF_good_calls"] = tropiseq_pred_serie_uf

merged_df_clean_01["TropiLogReg_good_calls"] = tropilr_pred_serie
merged_df_clean_01["SpikeHunter_good_calls"] = sh_pred_serie


merged_df_clean_01["Targets"] = target_serie
merged_df_clean_01["Folds"] = fold_serie

merged_df_clean_01.columns  = ['phage', 'protein', 'TropiGAT_prediction', 'TropiGAT_new_prediction',
       'TropiGAT_new_UF_prediction', 'TropiSAGE_prediction', 'TropiSEQ_prediction',
       'TropiSEQ_UF_prediction', 'TropiLogReg_prediction',
       'SpikeHunter_prediction', 'TropiGAT_good_calls',
       'TropiGAT_new_good_calls', 'TropiGAT_new_UF_good_calls',
       'TropiSAGE_good_calls', 'TropiSEQ_good_calls', 'TropiSEQ_UF_good_calls',
       'TropiLogReg_good_calls', 'SpikeHunter_good_calls', 'Targets', 'Folds']

In [37]:
merged_df_clean_01.columns

Index(['phage', 'protein', 'TropiGAT_prediction', 'TropiGAT_new_prediction',
       'TropiGAT_new_UF_prediction', 'TropiSAGE_prediction',
       'TropiSEQ_prediction', 'TropiSEQ_UF_prediction',
       'TropiLogReg_prediction', 'SpikeHunter_prediction',
       'TropiGAT_good_calls', 'TropiGAT_new_good_calls',
       'TropiGAT_new_UF_good_calls', 'TropiSAGE_good_calls',
       'TropiSEQ_good_calls', 'TropiSEQ_UF_good_calls',
       'TropiLogReg_good_calls', 'SpikeHunter_good_calls', 'Targets', 'Folds'],
      dtype='object')

In [124]:
#merged_df_clean_02 = merged_df_clean_01[["phage", "protein", "Folds", "TropiGAT_prediction", "TropiGAT_good_calls", "TropiGAT_new_prediction", "TropiGAT_new_good_calls", "TropiSAGE_prediction", "TropiSAGE_good_calls" , "TropiSEQ_prediction", "TropiSEQ_good_calls", "TropiLogReg_prediction", "TropiLogReg_good_calls" ,"SpikeHunter_prediction", "SpikeHunter_good_calls", "Targets"]]
merged_df_clean_02 = merged_df_clean_01[["phage", "protein", "Folds", "TropiGAT_prediction", "TropiGAT_good_calls", "TropiGAT_new_prediction", "TropiGAT_new_good_calls", "TropiGAT_new_UF_prediction", "TropiGAT_new_UF_good_calls", "TropiSAGE_prediction", "TropiSAGE_good_calls" , "TropiSEQ_prediction", "TropiSEQ_good_calls", "TropiSEQ_UF_prediction", "TropiSEQ_UF_good_calls","TropiLogReg_prediction", "TropiLogReg_good_calls" ,"SpikeHunter_prediction", "SpikeHunter_good_calls", "Targets"]]

merged_df_clean_02 = merged_df_clean_02.sort_values(by='phage', ascending=True)
merged_df_clean_02.to_csv(f"{path_benchmark}/Prediction_results.v4.matrices.tsv", sep = "\t", header = True, index = False)



***
### Get metrics:

In [125]:
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter 

path_benchmark = "/media/concha-eloko/Linux/PPT_clean/benchmark"

results_bench_df = pd.read_csv(f"{path_benchmark}/Prediction_results.v4.matrices.tsv", sep = "\t", index_col = False, header = 0)
results_bench_df["SpikeHunter_prediction"] = [kltype.replace("K", "KL") for kltype in results_bench_df["SpikeHunter_prediction"] if kltype.count("KL")==0]

In [126]:
results_bench_df

,phage,protein,Folds,TropiGAT_prediction,TropiGAT_good_calls,TropiGAT_new_prediction,TropiGAT_new_good_calls,TropiGAT_new_UF_prediction,TropiGAT_new_UF_good_calls,TropiSAGE_prediction,TropiSAGE_good_calls,TropiSEQ_prediction,TropiSEQ_good_calls,TropiSEQ_UF_prediction,TropiSEQ_UF_good_calls,TropiLogReg_prediction,TropiLogReg_good_calls,SpikeHunter_prediction,SpikeHunter_good_calls,Targets
0,A1a,A1a_00014,right-handed beta-helix,KL45:0.878 ; KL2:0.741 ; KL74:0.57 ; KL48:0.52...,0,KL26:0.962 ; KL70:0.961 ; KL140:0.632 ; KL36:0...,0,KL28:0.866 ; KL117:0.739 ; KL45:0.534,0,KL26:0.996 ; KL70:0.994 ; KL140:0.978 ; KL105:...,0,KL151:0.599 ; KL37:0.235 ; KL164:0.195 ; KL61:...,KL151,KL151:0.376 ; KL6:0.213 ; KL4:0.168 ; KL117:0....,KL151,No_associations,0,KL151,KL151,KL151
1,A1a,A1a_00002,6-bladed beta-propeller,KL123:0.977 ; KL14:0.911 ; KL13:0.9 ; KL39:0.8...,0,KL19:0.994 ; KL41:0.829 ; KL153:0.789 ; KL55:0...,0,KL123:1.0 ; KL4:0.996 ; KL55:0.985 ; KL124:0.9...,0,KL123:0.995 ; KL7:0.991 ; KL14:0.986 ; KL19:0....,0,KL102:0.737 ; KL7:0.448 ; KL37:0.235 ; KL164:0...,0,KL102:0.996 ; KL7:0.942 ; KL6:0.213 ; KL4:0.16...,0,No_associations,0,No_hits,0,KL151
2,A1b,A1b_00036,6-bladed beta-propeller,KL7:0.983 ; KL13:0.975 ; KL123:0.971 ; KL14:0....,0,KL19:0.987 ; KL7:0.966 ; KL43:0.873 ; KL153:0....,0,KL123:1.0 ; KL4:0.997 ; KL55:0.981 ; KL124:0.9...,0,KL7:0.999 ; KL123:0.994 ; KL14:0.987 ; KL43:0....,0,KL102:0.737 ; KL7:0.448 ; KL37:0.235 ; KL164:0...,0,KL102:0.996 ; KL7:0.942 ; KL6:0.213 ; KL4:0.16...,0,No_associations,0,No_hits,0,KL157
3,A1b,A1b_00048,right-handed beta-helix,KL128:0.919 ; KL70:0.885 ; KL52:0.862 ; KL46:0...,0,KL122:0.998 ; KL149:0.996 ; KL128:0.969 ; KL26...,KL157,KL28:0.961 ; KL71:0.959 ; KL157:0.927 ; KL155:...,KL157,KL157:0.999 ; KL70:0.997 ; KL152:0.996 ; KL26:...,KL157,KL157:0.469 ; KL37:0.235 ; KL164:0.195 ; KL61:...,KL157,KL155:0.56 ; KL6:0.213 ; KL4:0.168 ; KL117:0.1...,0,KL157:0.74,KL157,KL157,KL157,KL157
4,A1c,A1c_00034,right-handed beta-helix,KL52:0.998 ; KL62:0.993 ; KL128:0.981 ; KL43:0...,0,KL128:0.995 ; KL21:0.97 ; KL70:0.955 ; KL47:0....,0,KL128:0.971 ; KL21:0.872 ; KL43:0.812 ; KL34:0...,0,KL128:0.999 ; KL31:0.999 ; KL21:0.998 ; KL62:0...,0,No_hits,0,No_hits,0,No_hits,0,No_hits,0,KL1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,S13a,S13a_00036,right-handed beta-helix,KL18:0.959 ; KL27:0.938 ; KL60:0.87 ; KL123:0....,0,KL56:0.979 ; KL60:0.859 ; KL57:0.8 ; KL152:0.7...,0,KL57:0.947 ; KL60:0.779 ; KL122:0.707 ; KL53:0...,0,KL18:0.993 ; KL56:0.968 ; KL67:0.934 ; KL9:0.9...,0,KL38:0.38 ; KL37:0.235 ; KL164:0.195 ; KL61:0....,0,KL38:0.641 ; KL4:0.168 ; KL117:0.168 ; KL51:0....,0,KL38:0.741,0,KL38,0,"KL102,KL149"
235,S13b,S13b_00058,right-handed beta-helix,KL47:0.988 ; KL14:0.921 ; KL23:0.752 ; KL64:0....,0,KL47:0.996 ; KL152:0.95 ; KL64:0.715 ; KL70:0....,0,KL47:0.996 ; KL34:0.961 ; KL26:0.906 ; KL64:0....,0,KL28:1.0 ; KL112:1.0 ; KL152:0.999 ; KL8:0.993...,0,KL63:0.641 ; KL37:0.235 ; KL164:0.195 ; KL61:0...,KL63,KL63:0.583 ; KL6:0.213 ; KL4:0.168 ; KL117:0.1...,KL63,KL63:0.682,KL63,No_hits,0,KL63
236,S13c,S13c_00055,right-handed beta-helix,KL27:0.958 ; KL38:0.955 ; KL60:0.92 ; KL57:0.9...,0,KL56:0.991 ; KL38:0.986 ; KL57:0.915 ; KL43:0....,0,KL38:0.95 ; KL123:0.854 ; KL127:0.786 ; KL27:0...,0,KL56:0.995 ; KL127:0.995 ; KL38:0.995 ; KL18:0...,0,KL38:0.376 ; KL37:0.235 ; KL164:0.195 ; KL61:0...,0,KL38:0.374 ; KL6:0.213 ; KL4:0.168 ; KL117:0.1...,0,No_associations,0,KL38,0,"KL102,KL149"
237,S13d,S13d_00057,right-handed beta-helix,KL14:0.999 ; KL52:0.973 ; KL38:0.954 ; KL27:0....,KL14,KL14:0.991 ; KL43:0.91 ; KL21:0.906 ; KL152:0....,KL14,KL14:0.979 ; KL64:0.933 ; KL21:0.714 ; KL26:0....,KL14,KL14:1.0 ; KL152:0.994 ; KL21:0.992 ; KL43:0.8...,KL14,KL14:0.951 ; KL64:0.307 ; KL164:0.195 ; KL61:0...,KL14,KL14:0.823 ; KL64:0.437 ; KL6:0.213 ; KL4:0.16...,KL14,KL14:0.807,KL14,KL14,KL14,KL14


> Functions: 

In [10]:
# Useful functions: 

def make_labels(pred_df, predictor = "tropigat" , top_n = 40, to_ignore = []) :
    dico_pred = {"tropigat" : "TropiGAT_prediction" ,
                 "tropigat_uf" : "TropiGAT_new_UF_prediction", 
                 "tropiseq" : "TropiSEQ_prediction", 
                 "tropiseq_uf" : "TropiSEQ_UF_prediction", 
                 "spikehunter" : "SpikeHunter_prediction", 
                 "tropigat_new" : "TropiGAT_new_prediction", 
                 "tropilr" : "TropiLogReg_prediction", 
                 "tropisage" : "TropiSAGE_prediction"}
    col = dico_pred[predictor]
    labels_tropigat = {}
    all_kl_types = set(kltype.strip() for row in results_bench_df["Targets"] for kltype in row.split(","))
    count_kltypes = {}
    dico_prophage_count = all_kl_types
    for kltype in tqdm(dico_prophage_count) : 
        n = 0
        pred_labels , real_labels = [] , []
        for author in dico_matrices :
            if kltype in dico_matrices[author]["pool"] : 
                matrix = dico_matrices[author]["matrix"]
                for phage in matrix["Phages"].unique() :
                    #if phage not in to_ignore : 
                    if phage not in to_ignore and phage in pred_df["phage"].tolist(): 
                        top_predictions = set()
                        predictions = pred_df[pred_df["phage"] == phage][col].values
                        for calls in predictions : 
                            if predictor == "spikehunter" :
                                predicted_kltypes = [x.split(":")[0].strip().replace("K", "KL") for x in calls.split(";")]
                            else :
                                predicted_kltypes = [x.split(":")[0].strip() for x in calls.split(";")]
                            top_predictions.update(predicted_kltypes[0:top_n-1])
                        if kltype in top_predictions : 
                            pred_labels.append(1)
                        else : 
                            pred_labels.append(0)
                        if kltype in dico_hits[phage] : 
                            real_labels.append(1)
                            n += 1 
                        else :
                            real_labels.append(0)
        labels_tropigat[kltype] = {"y_pred" : pred_labels, "real_labels" : real_labels}
        sorted_dict = dict(sorted(labels_tropigat.items(), key=lambda item: int(item[0].split("KL")[1])))
        count_kltypes[kltype] = n 
    return sorted_dict , labels_tropigat, count_kltypes

def decript_dic(sorted_dict, labels_tropigat, count_kltypes, print_df=True):
    # Initialize lists to collect metrics when print_df is False
    kltypes, counts, f1_scores, accuracies, recalls, precisions, aucs, mccs = ([] for _ in range(8))
    
    if print_df:
        print("KL type", "Count", "F1", "Accuracy", "Recall", "Precision", "AUC", "MCC", sep="\t")
    
    for kltype in sorted_dict:
        if len(labels_tropigat[kltype]["real_labels"]) > 1:
            if Counter(labels_tropigat[kltype]["y_pred"])[1] == 0:
                if print_df:
                    print(kltype, count_kltypes[kltype], "error", sep="\t")
                else:
                    # Append NaN for each metric when Counter condition is met
                    kltypes.append(kltype)
                    counts.append(count_kltypes[kltype])
                    f1_scores.append(np.nan)
                    accuracies.append(np.nan)
                    recalls.append(np.nan)
                    precisions.append(np.nan)
                    aucs.append(np.nan)
                    mccs.append(np.nan)
            else:
                try:
                    # Calculate each metric
                    f1 = f1_score(labels_tropigat[kltype]["real_labels"], labels_tropigat[kltype]["y_pred"], average='binary')
                    precision = precision_score(labels_tropigat[kltype]["real_labels"], labels_tropigat[kltype]["y_pred"], average='binary')
                    recall = recall_score(labels_tropigat[kltype]["real_labels"], labels_tropigat[kltype]["y_pred"], average='binary')
                    mcc = matthews_corrcoef(labels_tropigat[kltype]["real_labels"], labels_tropigat[kltype]["y_pred"])
                    accuracy = accuracy_score(labels_tropigat[kltype]["real_labels"], labels_tropigat[kltype]["y_pred"])
                    auc = roc_auc_score(labels_tropigat[kltype]["real_labels"], labels_tropigat[kltype]["y_pred"])
                    
                    if print_df:
                        print(kltype, count_kltypes[kltype], round(f1, 5), round(accuracy, 5), round(recall, 5), 
                              round(precision, 5), round(auc, 5), round(mcc, 5), sep="\t")
                    else:
                        # Append each metric to the corresponding list
                        kltypes.append(kltype)
                        counts.append(count_kltypes[kltype])
                        f1_scores.append(f1)
                        accuracies.append(accuracy)
                        recalls.append(recall)
                        precisions.append(precision)
                        aucs.append(auc)
                        mccs.append(mcc)                        
                except ValueError:
                    if print_df:
                        print(kltype, "error", sep="\t")
                    else:
                        # Append NaN values when a ValueError occurs
                        kltypes.append(kltype)
                        counts.append(count_kltypes[kltype])
                        f1_scores.append(np.nan)
                        accuracies.append(np.nan)
                        recalls.append(np.nan)
                        precisions.append(np.nan)
                        aucs.append(np.nan)
                        mccs.append(np.nan)

    # Return lists when print_df is False
    if not print_df:
        dataframe = pd.DataFrame({
            "KL type": kltypes,
            "Count": counts,
            "F1": f1_scores,
            "Accuracy": accuracies,
            "Recall": recalls,
            "Precision": precisions,
            "AUC": aucs,
            "MCC": mccs
        })
        dataframe = dataframe[dataframe["Count"] > 0]
        return dataframe

    
def get_good_calls(df) :
    from collections import defaultdict
    
    n_cols_interest = 0
    # Make list of the predictions col names :
    cols_predictors = []
    for col in df.columns: 
        if col.count("good_calls") > 0 :
            cols_predictors.append(col)
    good_calls_dico = defaultdict(list)
    # iterate through the predictions : 
    for predictor in cols_predictors :
        good_calls_list = df[predictor].tolist()
        for phage, good_calls in zip(df["phage"].tolist(), good_calls_list) :
            if good_calls != "0" :
                for call in good_calls.split(",") :
                    a = (phage, call)
                    good_calls_dico[predictor].append(a)
    return good_calls_dico
                
    
def get_good_calls_extended(df, threshold = 15) :
    from collections import defaultdict
    
    n_cols_interest = 0
    # Make list of the predictions col names :
    cols_predictors = []
    for col in df.columns: 
        if col.count("good_calls") > 0 :
            cols_predictors.append(col)
    good_calls_dico = defaultdict(list)
    # iterate through the predictions : 
    for predictor in cols_predictors :
        good_calls_list = df[predictor].tolist()
        tool_pred = predictor.split("_good")[0]
        assert f"{tool_pred}_prediction" in df.columns, print(f"{tool_pred}_prediction not here ?")
        prediction_col = df[f"{tool_pred}_prediction"].tolist()
        for phage, fold, predictions, good_calls in zip(df["phage"].tolist(), df["Folds"].tolist(), prediction_col, good_calls_list) :
            if good_calls != "0" :
                pred_ranked = [pred.split(":")[0].strip() for pred in predictions.split(";")]
                for call in good_calls.split(",") :
                    try : 
                        rank = pred_ranked.index(call) + 1
                        if rank <= threshold :
                            a = (phage, call, fold, rank)
                            good_calls_dico[predictor].append(a)
                    except ValueError :
                        print("Error", call, pred_ranked, predictor)
    return good_calls_dico    

    
    

def clean_print(dico) :
	""" 
	Inputs : a dico
	Outputs : pretty printed dico
	"""
	import pprint
	pp = pprint.PrettyPrinter(width = 150, sort_dicts = True, compact = True)
	out = pp.pprint(dico)
	return out 
        

In [142]:
helices_results_bench_df = results_bench_df[results_bench_df["Folds"].isin(["triple-helix","right-handed beta-helix"])]

t = 10

good_calls_extended = get_good_calls_extended(results_bench_df, threshold = t)
good_calls_helices = get_good_calls_extended(helices_results_bench_df, threshold = t)

print("All Folds :")
for predictor in good_calls_extended :
    print(predictor, len(good_calls_extended[predictor]))
    
print("\nHelices Folds :")
for predictor in good_calls_helices :
    print(predictor, len(good_calls_helices[predictor]))
    


All Folds :
TropiGAT_good_calls 77
TropiGAT_new_good_calls 73
TropiGAT_new_UF_good_calls 86
TropiSAGE_good_calls 82
TropiSEQ_good_calls 97
TropiSEQ_UF_good_calls 86
TropiLogReg_good_calls 21
SpikeHunter_good_calls 44

Helices Folds :
TropiGAT_good_calls 61
TropiGAT_new_good_calls 63
TropiGAT_new_UF_good_calls 59
TropiSAGE_good_calls 66
TropiSEQ_good_calls 78
TropiSEQ_UF_good_calls 63
TropiLogReg_good_calls 21
SpikeHunter_good_calls 44


In [ ]:
clean_print(good_calls_extended)

In [ ]:
# Full dataset: 
sorted_dic_tropigat, labels_tropigat, count_kltypes_tropigat = make_labels(results_bench_df, predictor = "tropigat", top_n = 40)
sorted_dic_tropigat_n, labels_tropigat_n, count_kltypes_tropigat_n = make_labels(results_bench_df, predictor = "tropigat_new", top_n = 40)
sorted_dic_tropisage, labels_tropisage, count_kltypes_tropisage = make_labels(results_bench_df, predictor = "tropisage", top_n = 40)

sorted_dic_tropiseq, labels_tropiseq, count_kltypes_tropiseq = make_labels(results_bench_df, predictor = "tropiseq", top_n = 40)
sorted_dic_tropilr, labels_tropilr, count_kltypes_tropilr = make_labels(results_bench_df, predictor = "tropilr", top_n = 40)
sorted_dic_tropish, labels_tropish, count_kltypes_sh = make_labels(results_bench_df, predictor = "spikehunter", top_n = 40)

tropigat_full_df = decript_dic(sorted_dic_tropigat, labels_tropigat, count_kltypes_tropigat, print_df=False)
tropigat_n_full_df = decript_dic(sorted_dic_tropigat_n, labels_tropigat_n, count_kltypes_tropigat_n, print_df=False)
tropisage_full_df = decript_dic(sorted_dic_tropisage, labels_tropisage, count_kltypes_tropisage, print_df=False)

tropiseq_full_df = decript_dic(sorted_dic_tropiseq, labels_tropiseq, count_kltypes_tropiseq, print_df=False)
tropilr_full_df = decript_dic(sorted_dic_tropilr, labels_tropilr, count_kltypes_tropilr, print_df=False)
tropish_full_df = decript_dic(sorted_dic_tropish, labels_tropish, count_kltypes_sh, print_df=False)

tropigat_full_df.to_csv(f"{path_benchmark}/TropiGAT.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropigat_n_full_df.to_csv(f"{path_benchmark}/TropiGAT_new.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropisage_full_df.to_csv(f"{path_benchmark}/TropiSAGE.KL_metrics.full.tsv", sep = "\t", index = False, header = True)

tropiseq_full_df.to_csv(f"{path_benchmark}/TropiSEQ.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropilr_full_df.to_csv(f"{path_benchmark}/TropiLogReg.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropish_full_df.to_csv(f"{path_benchmark}/SpikeHunter.KL_metrics.full.tsv", sep = "\t", index = False, header = True)

> Comparative metrics, full: 

In [ ]:
comparative_full_df = pd.DataFrame({
            "KL type": tropigat_full_df["KL type"],
            "Count": tropigat_full_df["Count"],
            # recall
            "Recall_TropiGAT": tropigat_full_df["Recall"].round(3),
            "Recall_TropiGAT_n": tropigat_n_full_df["Recall"].round(3),
            "Recall_TropiSAGE": tropisage_full_df["Recall"].round(3),
            "Recall_TropiSEQ": tropiseq_full_df["Recall"].round(3),
            "Recall_TropiLogReg": tropilr_full_df["Recall"].round(3),
            "Recall_TropiSH": tropish_full_df["Recall"].round(3),
            # precision: 
            "Precision_TropiGAT": tropigat_full_df["Precision"].round(3),
            "Precision_TropiGAT_n": tropigat_n_full_df["Precision"].round(3),
            "Precision_TropiSAGE": tropisage_full_df["Precision"].round(3),
            "Precision_TropiLogReg": tropilr_full_df["Precision"].round(3),
            "Precision_TropiSEQ": tropiseq_full_df["Precision"].round(3),
            "Precision_TropiSH": tropish_full_df["Precision"].round(3),
            # MCC: 
            "MCC_TropiGAT": tropigat_full_df["MCC"].round(3),
            "MCC_TropiGAT_n": tropigat_n_full_df["MCC"].round(3),
            "MCC_TropiSAGE": tropisage_full_df["MCC"].round(3),
            "MCC_TropiLogReg": tropilr_full_df["MCC"].round(3),
            "MCC_TropiSEQ": tropiseq_full_df["MCC"].round(3),
            "MCC_TropiSH": tropish_full_df["MCC"].round(3),
        })

comparative_full_df = comparative_full_df.fillna("-")
comparative_full_df.to_csv(f"{path_benchmark}/Comparative.Full.tsv", sep = "\t", index = False, header = True)


In [ ]:
comparative_full_df

> Comparative metrics, helices:

In [ ]:
# Helices:  
helices_results_bench_df = results_bench_df[results_bench_df["Folds"].isin(["triple-helix","right-handed beta-helix"])]

sorted_dic_tropigat_helices, labels_tropigat_helices, count_kltypes_tropigat_helices = make_labels(helices_results_bench_df, predictor = "tropigat", top_n = 40)
sorted_dic_tropigat_helices, labels_tropigat_helices, count_kltypes_tropigat_helices = make_labels(helices_results_bench_df, predictor = "tropigat", top_n = 40)

sorted_dic_tropiseq_helices, labels_tropiseq_helices, count_kltypes_tropiseq_helices = make_labels(helices_results_bench_df, predictor = "tropiseq", top_n = 40)
sorted_dic_tropiseq_helices, labels_tropiseq_helices, count_kltypes_tropiseq_helices = make_labels(helices_results_bench_df, predictor = "tropiseq", top_n = 40)

sorted_dic_tropish_helices, labels_tropish_helices, count_kltypes_sh_helices = make_labels(helices_results_bench_df, predictor = "spikehunter", top_n = 40)

tropigat_helices_df = decript_dic(sorted_dic_tropigat_helices, labels_tropigat_helices, count_kltypes_tropigat_helices, print_df=False)
tropiseq_helices_df = decript_dic(sorted_dic_tropiseq_helices, labels_tropiseq_helices, count_kltypes_tropiseq_helices, print_df=False)
tropish_helices_df = decript_dic(sorted_dic_tropish_helices, labels_tropish_helices, count_kltypes_sh_helices, print_df=False)

tropigat_helices_df.to_csv(f"{path_benchmark}/TropiGAT.KL_metrics.helices.tsv", sep = "\t", index = False, header = True)
tropiseq_helices_df.to_csv(f"{path_benchmark}/TropiSEQ.KL_metrics.helices.tsv", sep = "\t", index = False, header = True)
tropish_helices_df.to_csv(f"{path_benchmark}/SpikeHunter.KL_metrics.helices.tsv", sep = "\t", index = False, header = True)


# Full dataset: 
sorted_dic_tropigat, labels_tropigat, count_kltypes_tropigat = make_labels(results_bench_df, predictor = "tropigat", top_n = 40)
sorted_dic_tropigat_n, labels_tropigat_n, count_kltypes_tropigat_n = make_labels(results_bench_df, predictor = "tropigat_new", top_n = 40)
sorted_dic_tropisage, labels_tropisage, count_kltypes_tropisage = make_labels(results_bench_df, predictor = "tropisage", top_n = 40)

sorted_dic_tropiseq, labels_tropiseq, count_kltypes_tropiseq = make_labels(results_bench_df, predictor = "tropiseq", top_n = 40)
sorted_dic_tropilr, labels_tropilr, count_kltypes_tropilr = make_labels(results_bench_df, predictor = "tropilr", top_n = 40)
sorted_dic_tropish, labels_tropish, count_kltypes_sh = make_labels(results_bench_df, predictor = "spikehunter", top_n = 40)

tropigat_full_df = decript_dic(sorted_dic_tropigat, labels_tropigat, count_kltypes_tropigat, print_df=False)
tropigat_n_full_df = decript_dic(sorted_dic_tropigat_n, labels_tropigat_n, count_kltypes_tropigat_n, print_df=False)
tropisage_full_df = decript_dic(sorted_dic_tropisage, labels_tropisage, count_kltypes_tropisage, print_df=False)

tropiseq_full_df = decript_dic(sorted_dic_tropiseq, labels_tropiseq, count_kltypes_tropiseq, print_df=False)
tropilr_full_df = decript_dic(sorted_dic_tropilr, labels_tropilr, count_kltypes_tropilr, print_df=False)
tropish_full_df = decript_dic(sorted_dic_tropish, labels_tropish, count_kltypes_sh, print_df=False)

tropigat_full_df.to_csv(f"{path_benchmark}/TropiGAT.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropigat_n_full_df.to_csv(f"{path_benchmark}/TropiGAT_new.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropisage_full_df.to_csv(f"{path_benchmark}/TropiSAGE.KL_metrics.full.tsv", sep = "\t", index = False, header = True)

tropiseq_full_df.to_csv(f"{path_benchmark}/TropiSEQ.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropilr_full_df.to_csv(f"{path_benchmark}/TropiLogReg.KL_metrics.full.tsv", sep = "\t", index = False, header = True)
tropish_full_df.to_csv(f"{path_benchmark}/SpikeHunter.KL_metrics.full.tsv", sep = "\t", index = False, header = True)

In [ ]:
comparative_helices_df = pd.DataFrame({
            "KL type": tropigat_helices_df["KL type"],
            "Count": tropigat_helices_df["Count"],
            # recall
            "Recall_TropiGAT": tropigat_helices_df["Recall"].round(3),
            "Recall_TropiSEQ": tropiseq_helices_df["Recall"].round(3),
            "Recall_TropiSH": tropish_helices_df["Recall"].round(3),
            # precision: 
            "Precision_TropiGAT": tropigat_helices_df["Precision"].round(3),
            "Precision_TropiSEQ": tropiseq_helices_df["Precision"].round(3),
            "Precision_TropiSH": tropish_helices_df["Precision"].round(3),
            # MCC: 
            "MCC_TropiGAT": tropigat_helices_df["MCC"].round(3),
            "MCC_TropiSEQ": tropiseq_helices_df["MCC"].round(3),
            "MCC_TropiSH": tropish_helices_df["MCC"].round(3),
        })

comparative_helices_df = comparative_helices_df.fillna("-")
comparative_helices_df.to_csv(f"{path_benchmark}/Comparative.Helices.tsv", sep = "\t", index = False, header = True)

In [ ]:
comparative_helices_df

***
### Experimentally validated depolymerase results: 

In [3]:
# regular filtration: 
#tropigat_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiGAT.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiGAT_prediction"])
tropigat_new_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiGAT_new.review.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiGAT_new_prediction"])
#tropigat_no_heads_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiGAT_no_heads.review.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiGAT_no_heads_prediction"])


tropisage_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiSAGE.review.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiSAGE_prediction"])

tropiseq_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiSEQ.p_05.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiSEQ_prediction"])
tropilr_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiLogReg.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiLR_prediction"])

# Ultra filtration: 
tropigat_new_pred_xp_df_uf = pd.read_csv(f"{path_benchmark}/TropiGAT_new_UF.review.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiGAT_UF_prediction"])
tropisage_pred_xp_df_uf = pd.read_csv(f"{path_benchmark}/TropiSAGE_UF.review.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiSAGE_UF_prediction"])

tropiseq_pred_xp_df_uf = pd.read_csv(f"{path_benchmark}/TropiSEQ_UF_70.p_05.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "TropiSEQ_UF_prediction"])
#tropilr_pred_xp_df = pd.read_csv(f"{path_benchmark}/TropiLogReg.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "prediction"])

# SpikeHunter
SH_pred_xp_df = pd.read_csv(f"{path_benchmark}/SpikeHunter_predictions.exp_val_depolymerase.tsv", sep = "\t", names = ["protein", "SpikeHunter_prediction"])
SH_pred_xp_df["SpikeHunter_prediction"] = SH_pred_xp_df["SpikeHunter_prediction"].apply(lambda x : x.replace("K", "KL"))


> Add metadata: 

In [4]:
info_depo = pd.read_csv(f"{path_benchmark}/exp_val_depolymerase.tsv", sep = "\t", header = 0)


In [5]:
#merged_df = tropigat_pred_xp_df.merge(tropigat_new_pred_xp_df, on='protein', how='inner').merge(tropisage_pred_xp_df, on='protein', how='inner').merge(tropiseq_pred_xp_df, on='protein', how='inner').merge(tropigat_new_pred_xp_df_uf, on='protein', how='inner').merge(tropisage_pred_xp_df_uf, on='protein', how='inner').merge(tropiseq_pred_xp_df_uf, on='protein', how='inner').merge(SH_pred_xp_df, on='protein', how='inner').merge(tropigat_no_heads_pred_xp_df, on='protein', how='inner').merge(tropilr_pred_xp_df, on='protein', how='inner')
#merged_df = tropigat_pred_xp_df.merge(tropigat_new_pred_xp_df, on='protein', how='inner').merge(tropisage_pred_xp_df, on='protein', how='inner').merge(tropiseq_pred_xp_df, on='protein', how='inner').merge(tropigat_new_pred_xp_df_uf, on='protein', how='inner').merge(tropisage_pred_xp_df_uf, on='protein', how='inner').merge(tropiseq_pred_xp_df_uf, on='protein', how='inner').merge(SH_pred_xp_df, on='protein', how='inner').merge(tropilr_pred_xp_df, on='protein', how='inner')
merged_df = tropigat_new_pred_xp_df.merge(tropisage_pred_xp_df, on='protein', how='inner').merge(tropiseq_pred_xp_df, on='protein', how='inner').merge(tropigat_new_pred_xp_df_uf, on='protein', how='inner').merge(tropisage_pred_xp_df_uf, on='protein', how='inner').merge(tropiseq_pred_xp_df_uf, on='protein', how='inner').merge(SH_pred_xp_df, on='protein', how='inner').merge(tropilr_pred_xp_df, on='protein', how='inner')


missing_protein = merged_df[~merged_df["protein"].isin(list(info_depo["protein"]))]["protein"]
merged_df = merged_df[merged_df["protein"].isin(list(info_depo["protein"]))]
target_col = []
phage_col = []

for _, row in merged_df.iterrows() :
    try : 
        target_col.append(info_depo[info_depo["protein"] == row["protein"]]["Target"].values[0])
        phage_col.append(info_depo[info_depo["protein"] == row["protein"]]["Phage"].values[0])
    except IndexError :
        print(row["protein"])


merged_df["Target"] = target_col
merged_df["Phage"] = phage_col




In [6]:
merged_df

len(merged_df)

34

In [193]:
merged_df[merged_df["protein"] == "APZ82768__116_426"]["TropiGAT_UF_prediction"].values

array(['KL18:0.931 ; KL2:0.916 ; KL60:0.89 ; KL45:0.874 ; KL155:0.832 ; KL71:0.825 ; KL122:0.758 ; KL10:0.666 ; KL30:0.46 ; KL35:0.395 ; KL116:0.38 ; KL9:0.343 ; KL36:0.307 ; KL34:0.291 ; KL3:0.241 ; KL151:0.226 ; KL105:0.18 ; KL23:0.172 ; KL20:0.165 ; KL5:0.149 ; KL169:0.146 ; KL64:0.129 ; KL153:0.12 ; KL1:0.093 ; KL56:0.063 ; KL63:0.041 ; KL43:0.04 ; KL46:0.039 ; KL117:0.038 ; KL4:0.035 ; KL109:0.035 ; KL128:0.029 ; KL31:0.028 ; KL15:0.027 ; KL48:0.024 ; KL149:0.022 ; KL26:0.02 ; KL19:0.018 ; KL124:0.018 ; KL67:0.015 ; KL110:0.013 ; KL25:0.013 ; KL14:0.011 ; KL81:0.01 ; KL106:0.01 ; KL8:0.01 ; KL103:0.008 ; KL112:0.008 ; KL13:0.008 ; KL142:0.008 ; KL136:0.007 ; KL157:0.005 ; KL55:0.005 ; KL102:0.005 ; KL52:0.004 ; KL51:0.004 ; KL111:0.004 ; KL17:0.004 ; KL24:0.004 ; KL28:0.004 ; KL27:0.003 ; KL127:0.003 ; KL16:0.003 ; KL145:0.003 ; KL12:0.003 ; KL74:0.002 ; KL39:0.002 ; KL114:0.002 ; KL6:0.002 ; KL47:0.002 ; KL108:0.002 ; KL125:0.001 ; KL118:0.001 ; KL107:0.001 ; KL70:0.001 ; KL21:0.

In [13]:

merged_df

,protein,TropiGAT_new_prediction,TropiSAGE_prediction,TropiSEQ_prediction,TropiGAT_UF_prediction,TropiSAGE_UF_prediction,TropiSEQ_UF_prediction,SpikeHunter_prediction,TropiLR_prediction,Target,Phage
0,AB716666.1_prot_BAP15736.1_24,KL19:0.991 ; KL41:0.851 ; KL153:0.749 ; KL6:0....,KL7:0.998 ; KL123:0.995 ; KL14:0.986 ; KL118:0...,KL102:0.737 ; KL7:0.448 ; KL37:0.235 ; KL164:0...,KL123:0.999 ; KL4:0.996 ; KL55:0.987 ; KL14:0....,KL14:1.0 ; KL123:1.0 ; KL4:0.999 ; KL43:0.991 ...,KL102:0.996 ; KL7:0.942 ; KL6:0.213 ; KL4:0.16...,No_hits,No_associations,ND,Klebsiella phage NTUH-K2044-K1-1
1,AB716666.1_prot_BAP15746.1_34,KL128:0.997 ; KL47:0.985 ; KL21:0.984 ; KL70:0...,KL128:1.0 ; KL31:1.0 ; KL21:0.999 ; KL70:0.998...,No_hits,KL128:0.99 ; KL43:0.955 ; KL21:0.899 ; KL34:0....,KL34:0.999 ; KL140:0.996 ; KL62:0.994 ; KL9:0....,No_hits,No_hits,No_hits,K1,Klebsiella phage NTUH-K2044-K1-1
2,AB897757.1_prot_BAQ02835.1_55,KL123:0.848 ; KL46:0.811 ; KL152:0.689 ; KL70:...,KL53:0.984 ; KL123:0.976 ; KL152:0.934 ; KL9:0...,KL124:0.486 ; KL37:0.235 ; KL164:0.195 ; KL61:...,KL46:0.952 ; KL64:0.746 ; KL26:0.709 ; KL27:0....,KL109:1.0 ; KL36:0.988 ; KL149:0.918 ; KL103:0...,KL124:0.649 ; KL6:0.213 ; KL4:0.168 ; KL117:0....,O1,No_associations,K11*,Klebsiella Phage ΦK64-1
3,AB897757.1_prot_BAQ02836.1_56,KL70:0.983 ; KL52:0.974 ; KL12:0.972 ; KL26:0....,KL70:1.0 ; KL26:0.999 ; KL125:0.981 ; KL35:0.9...,KL70:0.888 ; KL21:0.674 ; KL149:0.43 ; KL37:0....,KL26:0.832 ; KL118:0.753 ; KL169:0.679 ; KL127...,KL109:0.997 ; KL26:0.977 ; KL149:0.957 ; KL103...,KL70:0.983 ; KL149:0.408 ; KL21:0.22 ; KL6:0.2...,KL21,KL70:0.898,KN4†,Klebsiella Phage ΦK64-1
4,AB897757.1_prot_BAQ02837.1_57,KL128:0.993 ; KL70:0.928 ; KL81:0.748 ; KL139:...,KL19:0.997 ; KL70:0.996 ; KL128:0.994 ; KL153:...,KL123:0.711 ; KL37:0.235 ; KL164:0.195 ; KL61:...,KL71:0.916 ; KL151:0.908 ; KL26:0.846 ; KL81:0...,KL26:0.999 ; KL151:0.998 ; KL109:0.991 ; KL103...,KL6:0.213 ; KL4:0.168 ; KL117:0.168 ; KL51:0.1...,"KL123 , KL60",KL123:0.632,K21*,Klebsiella Phage ΦK64-1
5,AB897757.1_prot_BAQ02838.1_58,KL152:0.989 ; KL12:0.986 ; KL109:0.973 ; KL52:...,KL152:1.0 ; KL109:0.999 ; KL142:0.997 ; KL70:0...,No_hits,KL151:0.989 ; KL26:0.968 ; KL109:0.841 ; KL118...,KL128:1.0 ; KL151:1.0 ; KL142:0.998 ; KL26:0.9...,No_hits,No_hits,No_hits,KN5*,Klebsiella Phage ΦK64-1
6,AB897757.1_prot_BAQ02839.1_59,KL111:0.875 ; KL16:0.376 ; KL70:0.362 ; KL24:0...,KL111:0.979 ; KL29:0.977 ; KL18:0.893 ; KL19:0...,KL37:0.235 ; KL164:0.195 ; KL61:0.188 ; KL109:...,KL10:0.98 ; KL24:0.801 ; KL26:0.779 ; KL64:0.6...,KL26:0.999 ; KL109:0.997 ; KL10:0.996 ; KL111:...,KL25:0.496 ; KL6:0.213 ; KL4:0.168 ; KL117:0.1...,KL25,No_associations,K25†,Klebsiella Phage ΦK64-1
7,AB897757.1_prot_BAQ02840.1_60,KL36:1.0 ; KL19:0.975 ; KL52:0.967 ; KL152:0.9...,KL152:1.0 ; KL36:0.999 ; KL35:0.998 ; KL19:0.9...,KL35:0.66 ; KL37:0.235 ; KL164:0.195 ; KL61:0....,KL122:0.981 ; KL118:0.979 ; KL151:0.958 ; KL46...,KL128:1.0 ; KL36:0.999 ; KL26:0.999 ; KL151:0....,KL35:0.414 ; KL6:0.213 ; KL4:0.168 ; KL117:0.1...,KL35,No_associations,K35†,Klebsiella Phage ΦK64-1
8,AB897757.1_prot_BAQ02841.1_61,KL169:0.998 ; KL47:0.979 ; KL152:0.927 ; KL35:...,KL169:1.0 ; KL152:0.996 ; KL47:0.957 ; KL64:0....,No_hits,KL14:0.905 ; KL109:0.871 ; KL81:0.845 ; KL169:...,KL47:1.0 ; KL109:0.999 ; KL81:0.993 ; KL26:0.9...,No_hits,No_hits,No_hits,K1*,Klebsiella Phage ΦK64-1
9,AB897757.1_prot_BAQ02842.1_62,KL152:0.919 ; KL18:0.864 ; KL57:0.805 ; KL22:0...,KL152:1.0 ; KL112:1.0 ; KL14:0.994 ; KL18:0.99...,KL64:0.904 ; KL37:0.235 ; KL164:0.195 ; KL61:0...,KL26:0.986 ; KL64:0.984 ; KL151:0.966 ; KL109:...,KL155:1.0 ; KL103:1.0 ; KL64:1.0 ; KL109:1.0 ;...,KL64:0.374 ; KL6:0.213 ; KL4:0.168 ; KL117:0.1...,KL64,No_associations,K64*,Klebsiella Phage ΦK64-1


In [14]:
def get_target_rank(df):
    """
    This function calculates the rank of targets in predictions for each column in the dataframe.
    Inputs:
        - df: pandas DataFrame containing prediction columns and a 'Target' column.
    Outputs:
        - Dictionary with predictors as keys and target ranks as values.
    """
    from collections import defaultdict

    final_results = {}
    weird_char = ["*", "†"]
    targets_col = df['Target'].tolist()
    assert len(targets_col) == len(df), "Targets and predictions length mismatch!"

    # Identify prediction columns
    predictors = [col for col in df.columns if "prediction" in col]

    for predictor in predictors:
        dico_ranks = defaultdict(list)
        col_pred = df[predictor].tolist()

        for pred_list, target in zip(col_pred, targets_col):
            # Clean the target
            for char in weird_char:
                target = target.replace(char, "")
            if target == "ND" or "KN" in target:
                continue

            target_list = [
                t.replace("K", "KL") if "KL" not in t else t
                for t in target.split(",")
            ]

            # Check predictions
            if pred_list == "No_hits":
                for target in target_list:
                    dico_ranks[target].append("-")
            else:
                pred_ranked = [
                    pred.split(":")[0].strip() for pred in pred_list.split(";")
                ]
                for target in target_list:
                    if target in pred_ranked:
                        rank = pred_ranked.index(target) + 1  # 1-based index
                        dico_ranks[target].append(rank)
                    else:
                        dico_ranks[target].append("-")

        # Store ranks for the current predictor
        final_results[predictor] = dict(dico_ranks)

    return final_results

In [15]:
ranked_exp_depo = get_target_rank(merged_df)
clean_print(ranked_exp_depo)

{'SpikeHunter_prediction': {' KL69': ['-'],
                            'KL1': ['-', '-', '-', '-'],
                            'KL102': ['-'],
                            'KL11': ['-'],
                            'KL13': ['-'],
                            'KL2': [1, '-', '-'],
                            'KL21': ['-', 1],
                            'KL24': [1],
                            'KL25': [1],
                            'KL3': [1],
                            'KL30': [1],
                            'KL35': [1],
                            'KL47': [1, 1, 1, 1],
                            'KL63': ['-'],
                            'KL64': [1, 1, 1]},
 'TropiGAT_UF_prediction': {' KL69': ['-'],
                            'KL1': [19, 26, 23, 20],
                            'KL102': [36],
                            'KL11': ['-'],
                            'KL13': [49],
                            'KL2': [2, 59, 57],
                            'KL21': [61, 47],
         

In [16]:
df_exp = pd.DataFrame(ranked_exp_depo)
df_exp

,TropiGAT_new_prediction,TropiSAGE_prediction,TropiSEQ_prediction,TropiGAT_UF_prediction,TropiSAGE_UF_prediction,TropiSEQ_UF_prediction,SpikeHunter_prediction,TropiLR_prediction
KL1,"[59, 83, 59, 57]","[56, 70, 58, 47]","[-, -, -, -]","[19, 26, 23, 20]","[53, 41, 48, 58]","[-, -, -, -]","[-, -, -, -]","[-, -, -, -]"
KL11,[-],[-],[8],[-],[-],[-],[-],[-]
KL21,"[69, 65]","[91, 90]","[-, 2]","[61, 47]","[65, 68]","[-, 3]","[-, 1]","[-, -]"
KL25,[42],[48],[-],[33],[68],[1],[1],[-]
KL35,[12],[3],[1],[12],[17],[1],[1],[-]
KL64,"[7, 6, 1]","[16, 15, 13]","[1, 1, 1]","[2, 2, 1]","[3, 3, 3]","[1, 1, 1]","[1, 1, 1]","[-, -, -]"
KL30,[4],[9],[1],[9],[10],[-],[1],[-]
KL69,[-],[-],[-],[-],[-],[-],[-],[-]
KL47,"[1, 1, 1, 1]","[1, 1, 1, 1]","[1, 1, 1, 1]","[1, 1, 1, 1]","[1, 1, 1, 1]","[1, 1, 1, 1]","[1, 1, 1, 1]","[1, -, 1, -]"
KL2,"[16, 35, 37]","[16, 79, 77]","[1, -, -]","[2, 59, 57]","[20, 74, 74]","[1, -, -]","[1, -, -]","[-, -, -]"


In [17]:
merged_df.to_csv(f"{path_benchmark}/Metrics.exp_val_depolymerase.raw.tsv", sep = "\t", header = True, index = False)

df_exp.to_csv(f"{path_benchmark}/ranks.exp_val_depolymerase.raw.tsv", sep = "\t", header = True, index = True)


In [24]:
import numpy as np

data = {
    "TropiGAT_new_prediction": [[59, 83, 59, 57], [106], [69, 65], [42], [12], [7, 6, 1], [4], [1, 1, 1, 1], [16, 35, 37], [63], [1], [92], [53], [3]],
    "TropiSAGE_prediction": [[56, 70, 58, 47], [106], [91, 90], [48], [3], [16, 15, 13], [9], [1, 1, 1, 1], [16, 79, 77], [52], [2], [82], [40], [38]],
    "TropiSEQ_prediction": [[106, 106, 106, 106], [8], [106, 2], [106], [1], [1, 1, 1], [1], [1, 1, 1, 1], [1, 106, 106], [2], [1], [106], [1], [106]],
    "TropiGAT_UF_prediction": [[19, 26, 23, 20], [106], [61, 47], [33], [12], [2, 2, 1], [9], [1, 1, 1, 1], [2, 59, 57], [49], [2], [36], [23], [1]],
    "TropiSAGE_UF_prediction": [[53, 41, 48, 58], [106], [65, 68], [68], [17], [3, 3, 3], [10], [1, 1, 1, 1], [20, 74, 74], [66], [12], [49], [22], [4]],
    "TropiSEQ_UF_prediction": [[106, 106, 106, 106], [106], [106, 3], [1], [1], [1, 1, 1], [106], [1, 1, 1, 1], [1, 106, 106], [3], [2], [106], [1], [106]],
    "SpikeHunter_prediction": [[106, 106, 106, 106], [106], [106, 1], [1], [1], [1, 1, 1], [1], [1, 1, 1, 1], [1, 106, 106], [106], [1], [106], [106], [1]],
    "TropiLR_prediction": [[106, 106, 106, 106], [106], [106, 106], [106], [106], [106, 106, 106], [106], [1, 106, 1, 106], [106, 106, 106], [106], [1], [106], [1], [2]],
}

# KL types: The rows of the table
kl_types = [
    "KL1", "KL11", "KL21", "KL25", "KL35", "KL64", "KL30", "KL47",
    "KL2", "KL13", "KL24", "KL102", "KL63", "KL3"
]

# Function to calculate mean rank
def calculate_mean_rank(data, include_kl1=True):
    mean_ranks = {}
    for method, ranks in data.items():
        # Flatten ranks and filter for KL types if needed
        flattened = [rank for idx, row in enumerate(ranks) for rank in row]
        if not include_kl1:
            flattened = [rank for idx, row in enumerate(ranks) if kl_types[idx] != "KL1" for rank in row]
        # Compute mean rank if there are valid ranks
        mean_rank = np.mean(flattened) if flattened else None
        mean_ranks[method] = mean_rank

    return mean_ranks

# Compute mean ranks with and without KL1
mean_ranks_with_kl1 = calculate_mean_rank(data, include_kl1=True)
mean_ranks_without_kl1 = calculate_mean_rank(data, include_kl1=False)

# clean_print(mean_ranks_with_kl1)

clean_print(mean_ranks_without_kl1)



{'SpikeHunter_prediction': 36.0,
 'TropiGAT_UF_prediction': 24.095238095238095,
 'TropiGAT_new_prediction': 29.333333333333332,
 'TropiLR_prediction': 81.04761904761905,
 'TropiSAGE_UF_prediction': 31.80952380952381,
 'TropiSAGE_prediction': 37.19047619047619,
 'TropiSEQ_UF_prediction': 36.23809523809524,
 'TropiSEQ_prediction': 31.428571428571427}
